# Pulse Player Module

The pulse player is a simple module that plays analog pulses at the analog outputs.
Compared to the signal generator inside the Digital Unit Cells, the pulse player contains no additional digital upconversion but has storage for significantly longer pulses.

In [ ]:
import qiclib as ql
from qiclib.code import *

ip = "slot6-platform"

qic = ql.QiController(ip)

print(f"{len(qic.pulse_players)} pulse player(s) available")

In [ ]:
sample = QiSample(2)
for i in range(2):
    sample[i]["pi"] = 20e-9
    sample[i]["manip_frequency"] = 123e6

with QiJob() as flux_pulse:
    q = QiCells(1)
    c = QiCouplers(1)
    PlayFlux(c[0], QiPulse(length=4e-6, shape=ShapeLib.gauss))

flux_pulse.run(qic)

# Manual operation

For testing purposes, to change pulses in between experiments, or for other tasks where QiCode is not applicable, the hardware API can be used to update pulses on the platform manually.

The pulse players are accessible using the `pulse_players` property of the `QiController`.

Note that the actual count of usable pulses is one less than the count reported by `PulsePlayer.pulse_capactiy`. That is because there always is one implicit pulse with length zero.

In [ ]:
pp = qic.pulse_players[0]

print(f"The Pulse Player can hold up to {pp.pulse_capacity} pulses")

## Pulse access

Pulses are accessed using the `pulses` property of the `PulsePlayer`. You can assign most iterable, numeric values to the `pulses` property and also use more advanced assignments like slices or multiple indices.
Note that all set and get accesses communicate with the QiController.
Pulses obtained are always numpy arrays. Quantizing happens inside the `QiController`, pulses are expected to be within the (-1, 1) range.

In [ ]:
print(pp.pulses[:])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal

# Note that the following code will both send a gaussian pulse to the platform
# and also receive the same pulse from the platform for plotting purposes.
# This can, for example, be used to verify that the pulse is quantized back as expected.
t = np.arange(-200e-9, 200e-9, 0.5e-9)
pp.pulses[1] = signal.gausspulse(t, fc=10e6)

plt.plot(t * 1e9, pp.pulses[1])
plt.xlabel("Time [ns]")
plt.ylabel("Normalized Amplitude")
plt.show()

## Manual Triggering

Using this model, pulses must be triggered manually. Specify the pulse index and use the `PulsePlayer.trigger` method to play a pulse

In [ ]:
pp.trigger(1)

## Limitations

If you try to store a pulse that is too large, you will receive an error message. Note that this also applies when you store a pulse that would (on its own) fit into the available memory, but doesn't fit because there are other pulses that already take up space.

In [ ]:
pp.pulses[1] = [1] * 10_000